在 python 中，threading 模块是使用最多，也是官方推荐首选的线程模块。  

threading 模块提供了许多类，不同的类，具有不同的功能。 

其中，`threading.local` 是非常特别的一个类，我们接下来详细介绍它。 

# 一、threading.local() 类的理解
local 类，是 threading 模块中非常特别的一个类，用于表示 thread-local 数据。

## 1.1、什么是 thread-local 数据？
有人翻译为“线程本地数据”或“线程局部数据”，感觉都不是很形象，容易让人找不到北，我索性就用英文吧~  

**thread-local 数据，是特定的线程数据，用于多个线程之间的变量共享和数据隔离**。   

这里看似有些矛盾，既然是共享变量，为何又要数据隔离？ 

要回答这个问题，需要先了解局部变量和全局变量。  

全局变量和局部变量的区别在于作用域，全局变量在整个 .py 文件中声明，全局范围内可用； 
局部变量是在某个函数、类的内部声明，只能在函数、类内部使用，如果超出使用范围，则会报错。  

### 1.1.1、多线程中的局部变量

局部变量，是在某个函数、class内部声明的，只能在函数、class内部使用。  

多线程中，如果每个线程内部使用局部变量，互相不影响。  

例如： 

In [1]:
import threading


def func():
    # 声明局部变量 local_var
    local_var = 0
    for k in range(10):
        local_var += 1
    print(f'线程id：{threading.get_ident()}，局部变量：{local_var}')


# 创建3个线程，并启动
for th in range(3):
    threading.Thread(target=func).start()


线程id：25096，局部变量：10线程id：22524，局部变量：10

线程id：14984，局部变量：10


从上面的示例中，可以看到，每个线程内的局部变量 local_var 的值，都能正确累加，互不影响。

threading.Thread() 类，用于创建线程。  
threading.get_ident()方法，用于获取当前线程id。  

### 1.1.2、多线程中的全局变量

全局变量，在所有的函数、class外面，基于整个.py文件声明，全局范围内可以使用； 

使用global:  
如果想要在函数内为一个定义在函数外的全局变量赋值，那么就得告诉 python 这个变量名不是局部的，而是全局的，使用 global 语句可以完成这一功能。  
多线程中，如果多个线程使用同一全局变量，会互相影响。  

例如：

In [2]:
import threading
import time


# 声明全局变量 global_var
global_var = 0


def func():
    # 在函数内部，使用global语句，告知函数global_var为全局变量
    global global_var
    global_var = 0
    for k in range(100):
        # sleep一定时间，等待其他线程操作全局变量
        time.sleep(0.01)
        global_var += 1
    print(f'线程id：{threading.get_ident()}，全局变量：{global_var}')


# 创建3个线程，并启动
for th in range(3):
    threading.Thread(target=func).start()

线程id：19360，全局变量：298线程id：1904，全局变量：299
线程id：28724，全局变量：300



上面的例子中，当在主线程中声明全局变量 global_var 后，global_var 就变成了公共资源(也就是同一个对象)，每个子线程中使用 global_var，会互相干扰，最终导致了错误的计算结果。  

问：如何在不同线程中，使用同一个全局对象，并且让各个线程中该对象的值互不干扰呢？  

答：使用 thread-local 数据！  

上面说到：“thread-local 数据，是特定的线程数据，用于多个线程之间的变量共享和数据隔离。”  

既是共享变量，又能数据隔离，这是不矛盾的。  

thread-local 数据，允许多个线程共享一个变量对象，但是在各个线程内，该对象的属性值，相互独立。  

# 二、threading.local() 类的使用方法
我们再来接着讲 local 类的使用方法！  

local 类，是 threading 模块中，用于表示 thread-local 数据的类。  

通过实例化 local 类，得到一个 local 类的对象：  

```python
import threading  

local_obj = threading.local()  
```
在主线程中实例化一个 local 类的对象后，在不同的线程中使用这个对象存储的数据，其它线程不可见，即：各个线程中，该对象的值互不干扰。  

local类，本质上就是不同的线程使用这个对象时，为其创建一个独立的空间，存放数据。  

例如：

In [3]:
import threading
import time


# 利用local类，创建一个全局对象 local_obj
local_obj = threading.local()


def func():
    # 为local_obj对象声明一个属性 var
    local_obj.var = 0
    for k in range(100):
        # sleep一定时间，等待其他线程操作全局变量
        time.sleep(0.01)
        local_obj.var += 1
    print(f'线程id：{threading.get_ident()}，thread-local数据：{local_obj.var}')


# 创建3个线程，并启动
for th in range(3):
    threading.Thread(target=func).start()

线程id：20312，thread-local数据：100
线程id：22596，thread-local数据：100
线程id：12948，thread-local数据：100


上面的示例中，各个线程中使用同一对象 local_obj，但是 local_obj 对象在不同线程中的属性值互不干扰，均能正确输出计算结果。

需要注意，即使是在主线程中定义了 local 类的对象的属性，子线程中也访问不到主线程中的属性。  

对象属性，通过“.”取值；  
“键-值”通过“[]”取值，比如字典：dict_demo['key']。  

# 三、threading.local() 类的使用场景
这时候，大家也许会比较迷惑，  

如果需要数据隔离，使用局部变量，  

如果需要共享数据并同步，使用全局变量加锁就好了呀！  

为什么要有一个thread-local 数据？为什么要用 local() 类？local() 类，有什么使用场景呢？  

别急，我们接着慢慢看。  
  
在多线程环境下，每个线程都有自己的数据。   

所以，当我们使用线程的时候，能用线程的局部变量，就尽量不要用全局变量。  

因为使用全局变量涉及数据同步的问题，对于全局变量的修改必须加锁，保持各个线程中的数据同步。  



但是，有时候使用局部变量也不太方便。  

比如，多线程中涉及函数调用时，  

如果使用局部变量，那么必须通过参数传递变量；  

这时候如果使用全局变量也不行，因为每个线程处理的对象不同。  



这时候，使用 `threading.local`，就能发挥出作用了。  

在全局作用域，通过实例化 `threading.local` 类，得到一个 thread-local 数据对象，该thread-local 数据即为一个全局变量，但是每个线程却可以利用它来保存属于自己的私有属性，这些私有属性对其他线程也是不可见的。

通过调用全局对象的私有属性，自动获取当前线程对应的属性值，使代码更优雅简洁！

例如：

In [4]:
import threading
import time


# 利用local类，创建一个全局对象 local_obj
local_obj = threading.local()


def func():
    local_obj.var = 0
    # 如果使用局部变量，函数调用需要传参
    func_print()

def func_print():
    for k in range(100):
        time.sleep(0.01)
        # 直接使用local_obj.var，自动获取当前线程对应的属性值
        local_obj.var += 1
    print(f'线程id：{threading.get_ident()}，thread-local数据：{local_obj.var}')

# 创建3个线程，并启动
for th in range(3):
    threading.Thread(target=func,).start()

线程id：14040，thread-local数据：100线程id：19444，thread-local数据：100
线程id：28980，thread-local数据：100

